## Author: Ben Cliff

# About the data  
##### The dataset loaded into this notebook is a .csv file obtained from the CDC's preventative measures site.  
##### The sample subject of this dataset is U.S. citizens who are 65 years or older. The sample is subdivided by gender, State and city.
##### The target or signal of this dataset is the percentage of citizens (broken down by gender and location) who are reported to have taken preventative measures such as immunizations and select cancer screenings against common illnesses.
##### [Bethlehem insert commentary here about what trends we want to see]  

# Purpose of this notebook:
##### Create a clean .csv file to work from for further analysis
##### Perform preliminary analysis as well as diagnostics of the data
##### Create visualizations from this notebook for upload onto our blog site

# Reading and cleaning the dataset

In [1]:
# Importing powerful data manipulation library
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
# import shapely
from shapely.geometry import Point

In [2]:
# Reading in prevention data
prevent_df = pd.read_csv('../data/preventativedata.csv')

# Removing with little to no information
prevent_df = prevent_df.drop(columns=['Data_Value_Unit', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'TractFIPS', 'CategoryID', 'StateDesc', 'Data_Value_Type', 'DataSource',
'DataValueTypeID', 'Category'])

In [3]:
# Removing undescores
prevent_df.columns = prevent_df.columns.str.replace('_', '')
prevent_df.columns

Index(['Year', 'StateAbbr', 'CityName', 'GeographicLevel', 'UniqueID',
       'Measure', 'DataValue', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'PopulationCount', 'GeoLocation', 'MeasureId', 'CityFIPS',
       'ShortQuestionText'],
      dtype='object')

In [4]:
# Sorting values by unique ID; creating new index and dropping the old one
prevent_df.sort_values(by=['UniqueID'], ascending=True).reset_index().head(3).drop(columns=['index'])

,Year,StateAbbr,CityName,GeographicLevel,UniqueID,Measure,DataValue,LowConfidenceLimit,HighConfidenceLimit,PopulationCount,GeoLocation,MeasureId,CityFIPS,ShortQuestionText
0,2016,HI,Honolulu,City,15003,Older adult men aged >=65 Years who are up to ...,31.4,31.1,31.7,"953,207","(21.4588039305, -157.973296737)",COREM,15003,Core preventive services for older men
1,2016,HI,Honolulu,City,15003,Older adult women aged >=65 Years who are up t...,30.4,30.1,30.7,"953,207","(21.4588039305, -157.973296737)",COREW,15003,Core preventive services for older women
2,2016,AL,Birmingham,City,107000,Older adult men aged >=65 Years who are up to ...,32.9,32.2,33.7,"212,237","(33.5275663773, -86.7988174678)",COREM,107000,Core preventive services for older men


In [5]:
temp_list = []

longs = []

prevent_df['GeoLocation'] = [x.replace('(', '') for x in prevent_df['GeoLocation']]
prevent_df['GeoLocation'] = [x.replace(')', '') for x in prevent_df['GeoLocation']]
prevent_df['GeoLocation'] = [x.split(',') for x in prevent_df['GeoLocation']]
#prevent_df['GeoLocation'] = [float(x) for x in prevent_df['GeoLocation']]


In [6]:
lats = [float(x[0]) for x in prevent_df['GeoLocation']]
longs = [float(x[1]) for x in prevent_df['GeoLocation']]

In [7]:
prevent_df['lats'] = lats
prevent_df['longs'] = longs
prevent_df.head(5)

,Year,StateAbbr,CityName,GeographicLevel,UniqueID,Measure,DataValue,LowConfidenceLimit,HighConfidenceLimit,PopulationCount,GeoLocation,MeasureId,CityFIPS,ShortQuestionText,lats,longs
0,2016,WA,Yakima,City,5380010,Older adult men aged >=65 Years who are up to ...,31.3,29.6,33.0,"91,067","[46.5925792092, -120.547807614]",COREM,5380010,Core preventive services for older men,46.592579,-120.547808
1,2016,AZ,Avondale,City,404720,Older adult women aged >=65 Years who are up t...,27.5,26.6,28.3,"76,238","[33.3858367405, -112.323626619]",COREW,404720,Core preventive services for older women,33.385837,-112.323627
2,2016,AZ,Chandler,City,412000,Older adult women aged >=65 Years who are up t...,34.5,33.7,35.3,"236,123","[33.28318981, -111.852210033]",COREW,412000,Core preventive services for older women,33.283190,-111.852210
3,2016,AZ,Gilbert,City,427400,Older adult men aged >=65 Years who are up to ...,39.1,38.0,40.2,"208,453","[33.310149392, -111.746237467]",COREM,427400,Core preventive services for older men,33.310149,-111.746237
4,2016,AL,Montgomery,City,151000,Older adult men aged >=65 Years who are up to ...,33.6,32.6,34.6,"205,764","[32.3472645333, -86.2677059552]",COREM,151000,Core preventive services for older men,32.347265,-86.267706


# Data Visualization and Analysis

In [8]:
# Importing visualization tool
import plotly.graph_objects as go
import plotly.express as px

In [10]:
tmp_list = []
for x in prevent_df['StateAbbr']:
    if x == 'CA' or x == 'TX' or x == 'FL' or x == 'IL' or x == 'MI':
        tmp_list.append('yes')
    else:
        tmp_list.append('no')
prevent_df['topFiveState'] = tmp_list

In [12]:
# Extracting only the male records of the dataset to get the unique counts of surveys by cities
men_df = prevent_df.loc[prevent_df['MeasureId'] == 'COREM']

# Isolating the top 5 counts of cities in separate dataframe
top_5_states = men_df.loc[(men_df['StateAbbr'] == 'CA') | (men_df['StateAbbr'] == 'TX') | (men_df['StateAbbr'] == 'FL')
| (men_df['StateAbbr'] == 'IL') | (men_df['StateAbbr'] == 'MI')]

# Isolating all other 45 states in separate dataframe
other_states_df = men_df.loc[(men_df['StateAbbr'] != 'CA') & (men_df['StateAbbr'] != 'TX') & (men_df['StateAbbr'] != 'FL')
& (men_df['StateAbbr'] != 'IL') & (men_df['StateAbbr'] != 'MI')]

In [13]:
# Creating separate data for the histogram counts
x = other_states_df['StateAbbr']
y = top_5_states['StateAbbr']

# Creating graph object
fig = go.Figure()
fig.add_trace(go.Histogram(histfunc='count', x=x, name='State Participants'))
fig.add_trace(go.Histogram(histfunc='count', x=y, name='Top 5 States represented', marker_color='#330C73'))
fig.update_layout(title_text='Cities Count of Survey', xaxis_title_text='State Abbreviation', yaxis_title_text='Count')
fig.show()

In [15]:

geometry = [Point(xy) for xy in zip(lats, longs)]
gdf = GeoDataFrame(prevent_df, geometry=geometry)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world.loc[world['name'] == 'United States of America']

fig = px.scatter_geo(prevent_df, lat=prevent_df['lats'], lon=prevent_df['longs'], color=prevent_df['topFiveState'], width=800, height=400)
fig.update_layout(title = 'PLACES survey locations', geo_scope='usa', showlegend=False)
fig.show()

In [ ]:

geometry = [Point(xy) for xy in zip(lats, longs)]
gdf = GeoDataFrame(prevent_df, geometry=geometry)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world.loc[world['name'] == 'United States of America']

fig = px.scatter_geo(prevent_df, lat=top_5_states['lats'], lon=top_5_states['longs'])
fig.update_traces(marker=dict(color='red'))
fig.update_layout(title = 'PLACES survey locations of the top 5 most surveyed states', geo_scope='usa')
fig.show()